# Initialize Spark

In [1]:
import findspark
import os
findspark.init(spark_home='/opt/spark')
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-arm64/'
os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python3.11'
# os.environ['PYSPARK_PYTHON'] = './environment/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/datasaku/.venv/bin/python3'
# os.environ['PYSPARK_DRIVER_PYTHON'] = './environment/bin/python'

In [2]:
!python --version

Python 3.11.9


In [3]:
!ls -a

.		 README.md	   sakuflow.egg-info  util
..		 dagster.yaml	   spark-warehouse    workspace.yaml
.git		 flow		   test.ipynb
.gitignore	 pyproject.toml    test_local.ipynb
.python-version  requirements.txt  tmpr8phikge


In [4]:
# test edit
# based on https://mybinder.org/v2/gh/projectnessie/nessie-demos/main?filepath=notebooks/nessie-iceberg-demo-nba.ipynb
# 
# to run the spark with custom site_package
# zip -r site.zip .
import pyspark
from pyspark.sql import SparkSession
import os
import socket

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.82.0'
# os.environ['JAVA_HOME']='/usr/lib/jvm/java-11-openjdk-arm64'
conf = pyspark.SparkConf().setAll([
    # ip of spark master
    # ('spark.driver.bindAddress', '10.244.0.14')
    ('spark.driver.host', socket.gethostbyname(socket.gethostname()))
    #, ('spark.ui.proxyBase', os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'proxy/4040')
    # ('spark.submit.deployMode', 'client')
    # , ('spark.driver.port', '7077')
    , ('spark.app.name', 'test')
    , ('spark.master', "spark://spark-master.spark-dev.svc.cluster.local:7077")
    # , ('spark.jars.packages', 'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.82.0,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-aws:3.3.4')
    , ("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
    # cluster ip
    # , ('spark.hadoop.fs.s3a.endpoint', 'http://10.104.71.148:6544')
    , ('spark.hadoop.fs.s3a.endpoint', "http://minio-service.minio-dev.svc.cluster.local:6544")
    # for minio spark
    , ('spark.hadoop.fs.s3a.access.key','minio')
    , ('spark.hadoop.fs.s3a.secret.key', 'minio123')
    , ('spark.hadoop.fs.s3a.path.style.access', 'true')
    , ("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # for iceberg minio jdbc
    #, ('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebesrgSparkSessionExtensions')
    , ('spark.sql.catalog.nessie_catalog', 'org.apache.iceberg.spark.SparkCatalog')
    , ("spark.sql.catalog.nessie_catalog.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    , ('spark.sql.defaultCatalog', 'nessie_catalog')
    , ('spark.sql.catalog.nessie_catalog.warehouse', 's3a://iceberg/')
    # , ('spark.sql.catalog.my_catalog.type', 'jdbc')
    # ip of service postgre
    , ('spark.sql.catalog.nessie_catalog.uri', 'http://nessie-service.nessie-dev.svc.cluster.local:6788/api/v1')
    , ('spark.sql.catalog.nessie_catalog.ref', 'main')
    , ("spark.sql.catalog.nessie_catalog.authentication.type", 'NONE')
    # , ('spark.sql.catalog.my_catalog.jdbc.user', 'postgres')
    # , ('spark.sql.catalog.my_catalog.jdbc.password', 'postgres')
    # , ('spark.pyspark.driver.python', './environment/bin/python')
    # , ('spark.driver.python', './environment/bin/python')
    # , ("spark.archives", '/home/datasaku/pyspark_venv.tar.gz#environment') # not working
    , ("spark.submit.pyFiles", "/home/datasaku/.venv/lib/python3.11/site-packages/site.zip")
    ])
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

24/09/06 19:42:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Create Table Manual

https://iceberg.apache.org/docs/latest/spark-ddl/

In [6]:
spark.sql("LIST REFERENCES IN nessie_catalog").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|d24cf9d4e15c69257...|
+-------+----+--------------------+



In [6]:
spark.sql("CREATE NAMESPACE nessie_catalog.datasaku").show()

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `datasaku` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [7]:
spark.sql(
    """CREATE TABLE IF NOT EXISTS nessie_catalog.datasaku.salaries
            (Season STRING, Team STRING, Salary STRING, Player STRING)"""
)

24/09/06 19:54:24 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[]

In [8]:
spark.sql("INSERT INTO nessie_catalog.datasaku.salaries VALUES ('1', 'bulls', '50', 'kurdo'), ('2', 'bulls', '51', 'kurdo');")

DataFrame[]

In [9]:
spark.sql("select * from nessie_catalog.datasaku.salaries;").show()

+------+-----+------+------+
|Season| Team|Salary|Player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



24/09/08 05:35:58 ERROR TaskSchedulerImpl: Lost executor 0 on 10.244.1.207: worker lost: Not receiving heartbeat for 60 seconds


In [10]:
sdf.writeTo("nessie_catalog.datasaku.salaries").append()

NameError: name 'sdf' is not defined

In [11]:
sdf = spark.table("nessie_catalog.datasaku.salaries")

In [12]:
sdf_agg = sdf.groupBy('Team').count()
sdf_agg.createOrReplaceTempView("salaries_agg")

In [13]:
spark.sql("select * from salaries_agg").show()

+-----+-----+
| Team|count|
+-----+-----+
|bulls|    4|
+-----+-----+



In [14]:
spark.sql("drop table nessie_catalog.datasaku.salaries_agg")

DataFrame[]

In [15]:
sdf_agg.writeTo("nessie_catalog.datasaku.salaries_agg").create()

In [16]:
spark.sql("SHOW REFERENCE IN nessie_catalog").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|2f8f9bc41a8b1b963...|
+-------+----+--------------------+



In [17]:
spark.sql("SHOW TABLES IN nessie_catalog.datasaku").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
| datasaku|dagster_test|      false|
| datasaku|    salaries|      false|
| datasaku|salaries_agg|      false|
+---------+------------+-----------+



In [18]:
spark.sql("select * from nessie_catalog.datasaku.salaries_agg").show()

+-----+-----+
| Team|count|
+-----+-----+
|bulls|    4|
+-----+-----+



# Read From File

https://spark.apache.org/docs/2.2.1/sql-programming-guide.html#datasets-and-dataframes
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html

In [15]:
sdf = spark.read.csv('s3a://input/nz.csv', header=True)
sdf.show(5)

+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|Year|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name|   Variable_category|  Value|Industry_code_ANZSIC06|
+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H01|        Total income|Financial perform...|757,504|  ANZSIC06 division...|
|2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H04|Sales, government...|Financial perform...|674,890|  ANZSIC06 division...|
|2021|                    Level 1|               99999|      All industries|Dollars (

In [16]:
sdf.select(sdf.Year).show(5)

+----+
|Year|
+----+
|2021|
|2021|
|2021|
|2021|
|2021|
+----+
only showing top 5 rows



In [17]:
sdf.select('Year', 'Units').show(5)

+----+------------------+
|Year|             Units|
+----+------------------+
|2021|Dollars (millions)|
|2021|Dollars (millions)|
|2021|Dollars (millions)|
|2021|Dollars (millions)|
|2021|Dollars (millions)|
+----+------------------+
only showing top 5 rows



In [18]:
sdf.filter(sdf.Year == 2021).show(5)

+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|Year|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name|   Variable_category|  Value|Industry_code_ANZSIC06|
+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H01|        Total income|Financial perform...|757,504|  ANZSIC06 division...|
|2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H04|Sales, government...|Financial perform...|674,890|  ANZSIC06 division...|
|2021|                    Level 1|               99999|      All industries|Dollars (

In [19]:
import pyspark.sql.functions as fs

sdf.filter(fs.col('Year') == '2021').show(5)

+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|Year|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name|   Variable_category|  Value|Industry_code_ANZSIC06|
+----+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H01|        Total income|Financial perform...|757,504|  ANZSIC06 division...|
|2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H04|Sales, government...|Financial perform...|674,890|  ANZSIC06 division...|
|2021|                    Level 1|               99999|      All industries|Dollars (

## Join

In [20]:
sdf1 = sdf.alias('sdf1')
sdf2 = sdf.alias('sdf2')
sdf2 = sdf2.withColumnRenamed('Year', 'Year Test')
sdf2.show(5)

+---------+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|Year Test|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name|   Variable_category|  Value|Industry_code_ANZSIC06|
+---------+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+-------+----------------------+
|     2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H01|        Total income|Financial perform...|757,504|  ANZSIC06 division...|
|     2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H04|Sales, government...|Financial perform...|674,890|  ANZSIC06 division...|
|     2021|                    Level 1|               99999|

In [21]:
change_column = [x.replace(' ','_') for x in sdf2.columns]
sdf2 = sdf2.toDF(*change_column)
sdf2.show()

+---------+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+----------------------+
|Year_Test|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name|   Variable_category|    Value|Industry_code_ANZSIC06|
+---------+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+---------+----------------------+
|     2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H01|        Total income|Financial perform...|  757,504|  ANZSIC06 division...|
|     2021|                    Level 1|               99999|      All industries|Dollars (millions)|          H04|Sales, government...|Financial perform...|  674,890|  ANZSIC06 division...|
|     2021|                    Level 1|           

In [26]:
# important to setup alias again before joining
sdf2 = sdf2.alias('sdf2')
join = sdf1.join(sdf2, (sdf1.Year == sdf2.Year_Test) & (sdf1.Units == sdf2.Units))
join.show(5)

+----+---------------------------+--------------------+--------------------+------------------+-------------+-------------+--------------------+-------+----------------------+---------+---------------------------+--------------------+--------------------+------------------+-------------+--------------------+------------------+------+----------------------+
|Year|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|Variable_name|   Variable_category|  Value|Industry_code_ANZSIC06|Year_Test|Industry_aggregation_NZSIOC|Industry_code_NZSIOC|Industry_name_NZSIOC|             Units|Variable_code|       Variable_name| Variable_category| Value|Industry_code_ANZSIC06|
+----+---------------------------+--------------------+--------------------+------------------+-------------+-------------+--------------------+-------+----------------------+---------+---------------------------+--------------------+--------------------+------------------+--------

In [23]:
sdf2.select(sdf2.Year_Test).show(5)

+---------+
|Year_Test|
+---------+
|     2021|
|     2021|
|     2021|
|     2021|
|     2021|
+---------+
only showing top 5 rows



In [24]:
sdf2.select(fs.col('Year_Test')).show(5)

+---------+
|Year_Test|
+---------+
|     2021|
|     2021|
|     2021|
|     2021|
|     2021|
+---------+
only showing top 5 rows



In [25]:
sdf2.printSchema()

root
 |-- Year_Test: string (nullable = true)
 |-- Industry_aggregation_NZSIOC: string (nullable = true)
 |-- Industry_code_NZSIOC: string (nullable = true)
 |-- Industry_name_NZSIOC: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Variable_code: string (nullable = true)
 |-- Variable_name: string (nullable = true)
 |-- Variable_category: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Industry_code_ANZSIC06: string (nullable = true)



In [136]:
join.select(fs.col('sdf1.Year'), fs.col('sdf2.Year_Test')).show(5)

+----+---------+
|Year|Year_Test|
+----+---------+
|2021|     2021|
|2021|     2021|
|2021|     2021|
|2021|     2021|
|2021|     2021|
+----+---------+
only showing top 5 rows



In [77]:
join.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Industry_aggregation_NZSIOC: string (nullable = true)
 |-- Industry_code_NZSIOC: string (nullable = true)
 |-- Industry_name_NZSIOC: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Variable_code: string (nullable = true)
 |-- Variable_name: string (nullable = true)
 |-- Variable_category: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Industry_code_ANZSIC06: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Industry_aggregation_NZSIOC: string (nullable = true)
 |-- Industry_code_NZSIOC: string (nullable = true)
 |-- Industry_name_NZSIOC: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Variable_code: string (nullable = true)
 |-- Variable_name: string (nullable = true)
 |-- Variable_category: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Industry_code_ANZSIC06: string (nullable = true)



# iceberg

https://iceberg.apache.org/docs/latest/spark-queries/#dataframe

## snapshot

In [29]:
spark.sql("select * from nessie_catalog.datasaku.salaries.snapshots").show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2024-06-04 21:22:...|6974540300266715054|               NULL|   append|s3a://iceberg/dat...|{spark.app.id -> ...|
|2024-06-06 17:22:...|4394921702574654866|6974540300266715054|   append|s3a://iceberg/dat...|{spark.app.id -> ...|
|2024-06-07 05:36:...|3941761197276919739|4394921702574654866|   append|s3a://iceberg/dat...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+



In [30]:
spark.sql("select * from nessie_catalog.datasaku.salaries.history").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-06-04 21:22:...|6974540300266715054|               NULL|               true|
|2024-06-06 17:22:...|4394921702574654866|6974540300266715054|               true|
|2024-06-07 05:36:...|3941761197276919739|4394921702574654866|               true|
+--------------------+-------------------+-------------------+-------------------+



In [31]:
spark.sql("select * from nessie_catalog.datasaku.salaries.files").show()

+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|content|           file_path|file_format|spec_id|record_count|file_size_in_bytes|        column_sizes|        value_counts|   null_value_counts|nan_value_counts|        lower_bounds|        upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|    readable_metrics|
+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|      0|s3a://iceberg/dat...|    PARQUET|      0|           1|              1089|{1 -> 36, 2 -> 41...|{1 -> 1, 2 -> 1, ...|{1 -> 0, 2 -> 0, ...|              {

In [32]:
spark.sql("select * from nessie_catalog.datasaku.salaries VERSION AS OF 6974540300266715054").show()

+------+-----+------+------+
|Season| Team|Salary|Player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



In [34]:
spark.read.format('iceberg').load('nessie_catalog.datasaku.salaries').show()

+------+-----+------+------+
|Season| Team|Salary|Player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     1|bulls|    50| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     2|bulls|    51| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



In [35]:
spark.read.option('snapshot-id', 6974540300266715054).format('iceberg').load('nessie_catalog.datasaku.salaries').show()

+------+-----+------+------+
|Season| Team|Salary|Player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



# try package

In [1]:
!pip install -e /data/datasaku/datasaku

Obtaining file:///data/datasaku/datasaku
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for datasaku (pyproject.toml) ... done
  Created wheel for datasaku: filename=datasaku-0.0.8-0.editable-py3-none-any.whl size=3539 sha256=2ab5ea9fe23f9f678f8880fca43c8f4d4693db2e128e6e5d8ebc6f53129ae0d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-1qvst43u/wheels/7d/b0/f7/718884e02c821debbabd25201967bfb75892ce674d5f4eb086
Successfully built datasaku
  Attempting uninstall: datasaku
    Found existing installation: datasaku 0.0.8
    Uninstalling datasaku-0.0.8:
      Successfully uninstalled datasaku-0.0.8


In [12]:
!export PYSPARK_PYTHON=/home/datasaku/.venv/bin/python3
!export SPARK_PYTHONPATH=/home/datasaku/.venv/bin/python3

In [13]:
!python --version

Python 3.11.9


In [3]:
from datasaku import datasaku_spark
Spark = datasaku_spark.DatasakuSparkNessieMinioIceberg(
    spark_home = '/opt/spark'
    , minio_username = 'minio'
    , minio_password = 'minio123'
)

In [4]:
with Spark.spark_session_context() as spark_session:
    spark_session.sql("LIST REFERENCES IN nessie_catalog").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| dev|f501c935282dce964...|
| Branch|main|e6177d6f9a5659eed...|
+-------+----+--------------------+



In [5]:
spark = Spark.spark_session()
spark.sql("select * from nessie_catalog.datasaku.salaries").show()

24/06/20 17:53:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------+-----+------+------+
|Season| Team|Salary|Player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     1|bulls|    50| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



In [6]:
spark.stop()

In [7]:
# with Spark.spark_session_context() as spark_session:
#     spark_session.sql("DROP TABLE nessie_catalog.datasaku.salaries")

In [8]:
# with Spark.spark_session_context() as spark_session:
#     spark_session.sql("DROP NAMESPACE nessie_catalog.datasaku")

In [9]:
spark_session.sql("CREATE NAMESPACE nessie_catalog.datasaku")

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `datasaku` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [10]:
spark_session.sql(
    """CREATE TABLE IF NOT EXISTS nessie_catalog.datasaku.salaries
            (Season STRING, Team STRING, Salary STRING, Player STRING)"""
)

DataFrame[]

In [14]:
with Spark.spark_session_context() as spark_session:
    spark_session.sql("LIST REFERENCES IN nessie_catalog").show()
    spark_session.sql("INSERT INTO nessie_catalog.datasaku.salaries VALUES ('1', 'bulls', '50', 'kurdo'), ('2', 'bulls', '51', 'kurdo');")

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| dev|f501c935282dce964...|
| Branch|main|9833effe4eeee1416...|
+-------+----+--------------------+



In [15]:
with Spark.spark_session_context() as spark_session:
    spark_session.sql("select * from nessie_catalog.datasaku.salaries").show()

+------+-----+------+------+
|Season| Team|Salary|Player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



In [41]:
from datasaku import datasaku_spark
Spark = datasaku_spark.DatasakuSparkNessieMinioIceberg(
    spark_home = '/opt/spark'
    , minio_username = 'minio'
    , minio_password = 'minio123'
)
spark = Spark.spark_session()
sdf = spark.read.table("nessie_catalog.datasaku.salaries")
sdf

DataFrame[Season: string, Team: string, Salary: string, Player: string]

In [42]:
from pyspark.sql import SparkSession, functions as fs
sdf = Spark.standarized_column_name(sdf)
sdf = Spark.trim_all_column(sdf)

In [43]:
sdf.show()

+------+-----+------+------+
|season| team|salary|player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



In [48]:
spark = Spark.spark_session()
spark.sql('select * from nessie_catalog.datasaku.salaries.history').show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-06-15 18:47:...|7754386482845123890|               NULL|               true|
|2024-06-15 18:49:...|8095789292677777677|7754386482845123890|               true|
|2024-06-15 19:34:...|3138826593629674888|8095789292677777677|               true|
+--------------------+-------------------+-------------------+-------------------+



In [50]:
spark.stop()

In [45]:
spark.stop()

In [46]:
with Spark.spark_session_context() as spark_session:
    spark_session.sql("delete from nessie_catalog.datasaku.salaries").show()

++
||
++
++



In [16]:

with Spark.spark_session_context() as spark_session:
    spark_session.sql("select * from nessie_catalog.datasaku.salaries.files").show(truncate=False)

+-------+--------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+------------+------------------+------------------------------------+--------------------------------+--------------------------------+----------------+-----------------------------------------------------------------------+-----------------------------------------------------------------------+------------+-------------+------------+-------------+------------------------------------------------------------------------------------------------------------------+
|content|file_path                                                                                                                                         |file_format|spec_id|record_count|file_size_in_bytes|column_sizes                        |value_counts                    |null_value_counts               |nan_value_counts|lower_bounds                  

In [58]:
with Spark.spark_session_context() as spark_session:
    spark_session.sql("select * from nessie_catalog.datasaku.salaries").show()

+------+-----+------+------+
|Season| Team|Salary|Player|
+------+-----+------+------+
|     1|bulls|    50| kurdo|
|     2|bulls|    51| kurdo|
+------+-----+------+------+



In [10]:
spark_session.sql(
    """CREATE TABLE IF NOT EXISTS nessie_catalog.datasaku.salaries_test
            (Season STRING, Team STRING, Salary STRING, Player STRING)"""
)

Py4JJavaError: An error occurred while calling o200.sql.
: org.projectnessie.client.rest.NessieInternalServerException: Internal Server Error (HTTP/500): java.lang.RuntimeException: Unhandled SQL exception, caused by org.postgresql.util.PSQLException: Connection to datasaku-postgres-postgresql-ha-pgpool.psql-dev.svc.cluster.local:5433 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections., caused by java.net.ConnectException: Connection refused
	at org.projectnessie.client.rest.ResponseCheckFilter.checkResponse(ResponseCheckFilter.java:70)
	at org.projectnessie.client.rest.NessieHttpResponseFilter.filter(NessieHttpResponseFilter.java:29)
	at org.projectnessie.client.http.impl.jdk11.JavaRequest.lambda$executeRequest$1(JavaRequest.java:143)
	at java.base/java.util.ArrayList.forEach(ArrayList.java:1541)
	at java.base/java.util.Collections$UnmodifiableCollection.forEach(Collections.java:1085)
	at org.projectnessie.client.http.impl.jdk

In [ ]:
with Spark.spark_session_context() as spark_session:
    spark_session.sql("LIST REFERENCES IN nessie_catalog").show()
    spark_session.sql("INSERT INTO nessie_catalog.datasaku.salaries_test VALUES ('1', 'bulls', '50', 'kurdo'), ('2', 'bulls', '51', 'kurdo');")

# pyspark to iceberg with metadata

In [3]:
!pip install -e /data/datasaku/datasaku

Obtaining file:///data/datasaku/datasaku
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.0 MB/s eta 0:00:00 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [6]:
!pip install python-ulid


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import findspark
import os
findspark.init(spark_home='/opt/spark')
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-arm64/'
os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python3.11'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/datasaku/.venv/bin/python3'

In [2]:
import pendulum
pendulum.now().format('YYYYMMDD')

'20240730'

In [3]:
from datasaku import datasaku_spark
Spark = datasaku_spark.DatasakuSparkNessieMinioIceberg(
    spark_home = '/opt/spark'
    , minio_username = 'minio'
    , minio_password = 'minio123'
)

In [4]:
spark = Spark.spark_session()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/30 18:39:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
if spark._jsparkSession.catalog().tableExists("nessie_catalog.datasaku.dagster_test"):
    print("table exist")

24/07/30 18:40:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


table exist


In [6]:
'datasaku' in spark.sql("SHOW NAMESPACES").toPandas()['namespace'].to_list()

True

In [7]:
spark.sql("SHOW REFERENCE").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|905c18b3f72bc6a3d...|
+-------+----+--------------------+



In [5]:
sdf = spark.createDataFrame(
    [
        ("1", "chicago", "100", "kurang"),  # create your data here, be consistent in the types.
        ("2", "chicago", "200", "kurang"),
    ],
    ["Season", "Team", "Salary", "Player"]  # add your column names here
)
sdf.show()

+------+-------+------+------+
|Season|   Team|Salary|Player|
+------+-------+------+------+
|     1|chicago|   100|kurang|
|     2|chicago|   200|kurang|
+------+-------+------+------+



In [6]:
key_column = ['Team','Season','Player']
all_column = sdf.columns

## create key

In [7]:
import farmhash
def farmhash_python(string:str):
    return farmhash.hash64(string)

In [8]:
import pyspark.sql.functions as fs
from pyspark.sql.types import StringType
farmhash_pyspark = fs.udf(lambda x: farmhash_python(x),StringType())

In [9]:
sdf.withColumn('test', fs.concat_ws('-', *key_column)).show()

+------+-------+------+------+----------------+
|Season|   Team|Salary|Player|            test|
+------+-------+------+------+----------------+
|     1|chicago|   100|kurang|chicago-1-kurang|
|     2|chicago|   200|kurang|chicago-2-kurang|
+------+-------+------+------+----------------+



In [10]:
sdf = sdf.withColumn('__key', farmhash_pyspark(fs.concat_ws('', *key_column)))
sdf.show()

+------+-------+------+------+--------------------+
|Season|   Team|Salary|Player|               __key|
+------+-------+------+------+--------------------+
|     1|chicago|   100|kurang|14633329190751059452|
|     2|chicago|   200|kurang|11836422410755210579|
+------+-------+------+------+--------------------+



## create unique_key

In [11]:
sdf = sdf.withColumn('__unique_row_id', fs.md5(fs.concat_ws('', *all_column)))
sdf.show()

+------+-------+------+------+--------------------+--------------------+
|Season|   Team|Salary|Player|               __key|     __unique_row_id|
+------+-------+------+------+--------------------+--------------------+
|     1|chicago|   100|kurang|14633329190751059452|375ff507bc5001fca...|
|     2|chicago|   200|kurang|11836422410755210579|8ab1342d6a8fa238e...|
+------+-------+------+------+--------------------+--------------------+



## create timestamp utc

In [12]:
import pendulum
sdf = sdf.withColumn('__created_timestamp_utc', fs.lit(pendulum.now('utc').to_iso8601_string()))
sdf.show(truncate=False)

+------+-------+------+------+--------------------+--------------------------------+---------------------------+
|Season|Team   |Salary|Player|__key               |__unique_row_id                 |__created_timestamp_utc    |
+------+-------+------+------+--------------------+--------------------------------+---------------------------+
|1     |chicago|100   |kurang|14633329190751059452|375ff507bc5001fca4d66619089e4455|2024-07-31T06:50:46.412475Z|
|2     |chicago|200   |kurang|11836422410755210579|8ab1342d6a8fa238ed09af412394aeca|2024-07-31T06:50:46.412475Z|
+------+-------+------+------+--------------------+--------------------------------+---------------------------+



## is_current_flag

In [13]:
sdf = sdf.withColumn('_is_current', fs.lit(True))
sdf.show()

+------+-------+------+------+--------------------+--------------------+-----------------------+-----------+
|Season|   Team|Salary|Player|               __key|     __unique_row_id|__created_timestamp_utc|_is_current|
+------+-------+------+------+--------------------+--------------------+-----------------------+-----------+
|     1|chicago|   100|kurang|14633329190751059452|375ff507bc5001fca...|   2024-07-31T06:50:...|       true|
|     2|chicago|   200|kurang|11836422410755210579|8ab1342d6a8fa238e...|   2024-07-31T06:50:...|       true|
+------+-------+------+------+--------------------+--------------------+-----------------------+-----------+



## ulid

In [14]:
from ulid import ULID

batch_id = str(ULID())

sdf = sdf.withColumn('_batch_id', fs.lit(batch_id))
sdf.show()

+------+-------+------+------+--------------------+--------------------+-----------------------+-----------+--------------------+
|Season|   Team|Salary|Player|               __key|     __unique_row_id|__created_timestamp_utc|_is_current|           _batch_id|
+------+-------+------+------+--------------------+--------------------+-----------------------+-----------+--------------------+
|     1|chicago|   100|kurang|14633329190751059452|375ff507bc5001fca...|   2024-07-31T06:50:...|       true|01J43RWGS0HZ0CBMC...|
|     2|chicago|   200|kurang|11836422410755210579|8ab1342d6a8fa238e...|   2024-07-31T06:50:...|       true|01J43RWGS0HZ0CBMC...|
+------+-------+------+------+--------------------+--------------------+-----------------------+-----------+--------------------+



## ulid test

In [20]:
from ulid import ULID
def ulid_python():
    return str(ULID())

import pyspark.sql.functions as fs
from pyspark.sql.types import StringType
ulid_pyspark = fs.udf(lambda : ulid_python(),StringType())

sdf = sdf.withColumn('__batch_id', ulid_pyspark())
sdf.show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o128.get

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
sdf.withColumn('test', fs.concat_ws('-', *key_column)).show()

In [22]:
import pendulum
def pendulum_now_python(string:str):
    return pendulum.now().to_date_string()

In [23]:
import pyspark.sql.functions as fs
from pyspark.sql.types import StringType
pendulum_now_pyspark = fs.udf(lambda x: pendulum_now_python(x),StringType())

In [24]:
sdf.withColumn('Timestamp', pendulum_now_pyspark(fs.col('Team'))).show()

24/07/13 21:08:46 WARN TaskSetManager: Lost task 0.0 in stage 14.0 (TID 13) (10.244.0.24 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 529, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = se

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 529, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
            

In [4]:
## check table

In [5]:
spark.read.format('iceberg').load('nessie_catalog.datasaku.dagster_test').show()

24/07/16 20:52:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------------+----------+------+-------+------+------+--------------------+-----------------------+------------+--------------+-------------------+
|Unnamed: 0.1|Unnamed: 0|Season|   Team|Salary|Player|     __unique_row_id|__created_timestamp_utc|__is_current|__logical_date|test_staging_column|
+------------+----------+------+-------+------+------+--------------------+-----------------------+------------+--------------+-------------------+
|           0|         0|     1|chicago|   100|kurang|375ff507bc5001fca...|   2024-07-07T12:54:...|        true|      20240707|test_staging_column|
|           0|         0|     1|chicago|   100|kurang|375ff507bc5001fca...|   2024-07-14T17:38:...|        true|      20240702|test_staging_column|
|           0|         0|     1|chicago|   100|kurang|375ff507bc5001fca...|   2024-07-13T11:03:...|        true|      20240708|test_staging_column|
|           0|         0|     1|chicago|   100|kurang|375ff507bc5001fca...|   2024-07-16T20:47:...|        true|

24/07/16 21:18:16 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/07/16 21:18:17 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [15]:
spark.sql("drop table nessie_catalog.datasaku.dagster_test")

24/07/31 06:51:47 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[]

24/07/31 07:12:58 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/07/31 07:12:59 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce